In [ ]:
#### NOTE TO SELF ######
# Linear behavior global vs braid

# SPC25 IS GOOD EXAMPLE
# Cenpe good
# Mgst3 maybe
# Irf8 maybe
# Ctsg is marginal




# import matplotlib as mpl
# mpl.rcParams.keys()

In [ ]:
import matplotlib as mpl
# COLOR = 'white'
# mpl.rcParams['text.color'] = COLOR
# mpl.rcParams['axes.labelcolor'] = COLOR
# mpl.rcParams['xtick.color'] = COLOR
# mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['figure.dpi'] = 100


# import fancyimpute as fi
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import variation
from math import isnan

from scipy.cluster.hierarchy import dendrogram,linkage

import sys
sys.path.append("../src/")
import lumberjack
import tree_reader as tr

import sklearn

## Table of Contents

* [Forest Generation](#forest_generation)
* [Prediction Error Analysis](#prediction_error)
* [Sample Clustering](#sample_clustering)
* [Split Clustering](#split_clustering)

## Forest Generation <a class="anchor" id="forest_generation"></a>

In [ ]:
# counts = np.loadtxt('/Users/boris/taylor/johnston_retina/single_cell/dmel-retina-scRNA/exploration/2018.07.19_Scanpy/log_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/johnston_retina/single_cell/dmel-retina-scRNA/exploration/2018.07.19_Scanpy/header.txt",dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/work/johnston_log_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/work/johnston_header.txt",dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/aging_sc/var_filtered_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/aging_sc/var_filtered_header.txt",dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/fan_tendon/log_counts.txt')
# header = np.loadtxt("/Users/boris/taylor/fan_tendon/header.txt",dtype=str)

# counts = np.loadtxt('/Users/bbrener1/taylor/raw_data/nesterowa/nesterowa_counts.txt')
# header = np.loadtxt('/Users/bbrener1/taylor/raw_data/nesterowa/nesterowa_gene_header.txt',dtype=str)

raw_counts = np.loadtxt('nesterowa_counts.txt')
raw_header = np.loadtxt('nesterowa_gene_header.txt',dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/nelmari/nc_filtered_log.txt')
# header = np.loadtxt('/Users/boris/taylor/nelmari/gene_header_filtered.txt',dtype=str)

# counts = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/testing/iris.trunc')
# header = np.loadtxt('/Users/boris/taylor/vision/rust_prototype/rusty_lumberjack/testing/iris.features',dtype=str)

# raw_counts = np.loadtxt('./citeseq_cbmc_counts.tsv')
# raw_header = np.loadtxt('./citeseq_cbmc_header.txt',dtype=str)

# raw_counts = np.loadtxt('./citeseq_cbmc_umis.tsv'')
                    

In [ ]:

feature_sort = dendrogram(linkage(raw_counts.T,metric='correlation',method='average'),no_plot=True)['leaves']
cell_sort = dendrogram(linkage(raw_counts,metric='cos',method='average'),no_plot=True)['leaves']

# feature_sort = np.arange(raw_counts.shape[1])
# cell_sort = np.arange(raw_counts.shape[0])

counts = raw_counts[cell_sort].T[feature_sort].T
counts = raw_counts.T[feature_sort].T
header = raw_header[feature_sort]

plt.figure(figsize=(15,10))
plt.title("Cell x Gene Expression Unsorted",fontsize=20)
plt.imshow(counts,aspect='auto',cmap="bwr")
plt.xlabel("Genes",fontsize=15)
plt.ylabel("Cells",fontsize=15)
plt.colorbar()
plt.show()


plt.figure(figsize=(15,10))
plt.title("Cell x Gene Expression, Agglomerative",fontsize=20)
plt.imshow(counts[cell_sort],aspect='auto',cmap="bwr")
plt.xlabel("Genes",fontsize=15)
plt.ylabel("Cells",fontsize=15)
plt.colorbar()
plt.show()

plt.figure()
plt.title("Frequency of Mean Gene Expression Values")
plt.xlabel("Mean Expression (log TPM)")
plt.ylabel("Frequency")
plt.hist(np.mean(counts,axis=0),bins=50,log=True)
plt.show()

plt.figure()
plt.title("Frequency of Individual Expression Values")
plt.xlabel("Mean Expression (log TPM)")
plt.ylabel("Frequency")
plt.hist(counts.flatten(),bins=50,log=True)
plt.show()


print(np.mean(counts,axis=0).shape)
print(header.shape)

In [ ]:
output_directory = "../prediction/"
# raw_text_out = open(output_directory + str("evaluation.txt"),mode='w')


In [ ]:

training_mask = np.random.random(counts.shape[0]) > .1
testing_mask = np.logical_not(training_mask)

training_counts = counts[training_mask]
testing_counts = counts[testing_mask]

# training_counts = counts 
# testing_counts = counts


# counts = sklearn.preprocessing.scale(counts)

forest = lumberjack.fit(
    training_counts,
    header=header,
    test_counts=testing_counts,
    trees=100,
    dispersion_mode="ssme",
    norm="l1",
    drop='none',
    sfr=0.5,
    in_feature_subsample=2000,
    out_feature_subsample=2000,
    sample_subsample=400,
    depth=8,
    leaves=50,
)


training_counts = forest.output
testing_counts = forest.test

true_counts = testing_counts

subsampling = 1

# mask = np.loadtxt('./testing/holdout_mask_counts.txt')

# held_out_counts = np.loadtxt('./testing/held_out_counts.txt')

#2 trees at 2:17



In [ ]:
# forest.weigh_leaves()
predicted = forest.predict_matrix(testing_counts,weighted=False)

# nodes = forest.predict_vector_nodes(forest.output[600])
# len(nodes)

# for sample in forest.output:
#     forest.predict_vector_leaves(sample)

# nodes[1].filter.reduction.features
# for node in nodes:
#     print(600 in node.samples)

In [ ]:
test_sort = dendrogram(linkage(true_counts,metric='cos',method='average'),no_plot=True)['leaves']
plt.figure()
plt.imshow(predicted[test_sort],aspect='auto')
plt.show()

plt.figure()
plt.imshow(true_counts[test_sort],aspect='auto')
plt.show()



In [ ]:

# forest = tr.Forest.reconstitute('./forest_nelmari')

forest = tr.Forest.reconstitute('./forest_vision')
# forest = tr.Forest.reconstitute('./forest_vision_lrg_l1')
# forest = tr.Forest.reconstitute('./forest_vision_lrg_l2')


# forest = tr.Forest.reconstitute('./forest_johnston_braid')

# forest = tr.Forest.load("../testing/nelmari/",input='../../../../../nelmari/nc_filtered_log.txt',output='../../../../../nelmari/nc_filtered_log.txt')
# forest = tr.Forest.load("../testing/nesterowa_forest/",input="../../work/nesterowa_counts.txt",output="../../work/nesterowa_counts.txt")
# forest = tr.Forest.load("../testing/johnston_forest/",input="../../work/johnston_log_counts.txt",output="../../work/johnston_log_counts.txt")


In [ ]:
# forest.backup('./forest_johnston_ihmm')
# forest.backup('./forest_johnston_ssme')
# forest.backup('./forest_johnston_flat_sme')
# forest.backup('./forest_johnston_var')
# forest.backup('./forest_fan_ssme')

# forest.backup('./forest_johnston_braid')

# forest.backup('./forest_nelmari')

forest.backup('./forest_vision')
# forest.backup('./forest_vision_small')
# forest.backup('./forest_vision_l1')
# forest.backup('./forest_vision_l2')
# forest.backup('./forest_vision_lrg')



In [ ]:
# Optional adjustment to truncate lower-expressing genes.
# expression_level_mask = np.mean(true_counts,axis=0) > 1
# true_counts = true_counts.T[expression_level_mask].T
# predicted = predicted.T[expression_level_mask].T

In [ ]:


# raw_text_out.write("=================================================\n")
# raw_text_out.write("Basic evaluation: \n")

# raw_text_out.write("Pearson R\n")
# raw_text_out.write(str(pearsonr(predicted.flatten(),true_counts.flatten())) + "\n")

# raw_text_out.write("MSE\n")
# raw_text_out.write(str(np.mean((predicted.flatten() - true_counts.flatten()) ** 2)) + "\n")

# raw_text_out.write("MAE\n")
# raw_text_out.write(str(np.mean(np.abs(predicted.flatten() - true_counts.flatten()))) + "\n")


# raw_text_out.close()

In [ ]:
# test_counts = forest.test

# prediction = forest.predict_matrix(test_counts)


In [ ]:
# sys.path.append("/Users/boris/haxx/python/smooth_density_graph/")
# import smooth_density_graph as sdg

# forest.reset_clusters()
# from scipy.cluster.hierarchy import linkage,dendrogram

# feature_sort = dendrogram(linkage(counts.T,metric='correlation',method='average'),no_plot=True)['leaves']

# counts = forest.output

encoding = forest.node_sample_encoding(forest.leaves())
# print(encoding.shape)

# print(np.sum(np.sum(encoding,axis=1) == 0))
cell_encoding_sort = dendrogram(linkage(encoding,metric='cos',method='average'),no_plot=True)['leaves']
leaf_sort = dendrogram(linkage(encoding.T,metric='cos',method='average'),no_plot=True)['leaves']
# feature_sort = dendrogram(linkage(forest.output.T,metric='cos',method='average'),no_plot=True)['leaves']
# feature_sort = np.argsort(np.var(forest.output,axis=0))


plt.figure(figsize=(10,10))
# plt.imshow(encoding,aspect='auto',cmap='binary')
plt.imshow(encoding[cell_encoding_sort].T[leaf_sort].T,aspect='auto',cmap='binary')
plt.show()


# # print(np.sum(np.isnan(encoding).flatten()))

# cell_clusterings = forest.cluster_samples_encoding(distance='cos',k=10,steps=50,override=False,verbose=True)
# # cell_clusterings = forest.cluster_samples_simple(pca=True,subsample=1400,distance='cos',k=10,steps=50,override=True,verbose=True)
# leaf_clusterings = forest.cluster_leaf_samples(distance='cos',k=10,steps=50,override=False,verbose=True)

# cell_order = np.argsort(cell_clusterings)
# leaf_order = np.argsort(leaf_clusterings)

# # clustered_counts = forest.output[cell_order].T[feature_sort].T
# clustered_counts = counts[cell_order].T[feature_sort].T
# clustered_encoding = encoding[cell_order].T[leaf_order].T

# plt.figure(figsize=(15,10))
# plt.title("Cell x Gene Expression, Forest Clustering",fontsize=20)
# plt.imshow(clustered_counts,aspect='auto')
# plt.xlabel("Genes",fontsize=15)
# plt.ylabel("Cells",fontsize=15)
# plt.colorbar()
# plt.show()

# plt.figure(figsize=(15,10))
# plt.title("Cell x Node Organization, Forest Clustering",fontsize=20)
# plt.imshow(clustered_encoding,aspect='auto',cmap='binary')
# plt.xlabel("Leaves",fontsize=15)
# plt.ylabel("Cells",fontsize=15)
# plt.show()

# print("===================")

# for cluster in forest.sample_clusters:
#     print(len(cluster.samples))

# print("===================")

# for cluster in forest.leaf_clusters:
#     print(len(cluster.nodes))

# print("===================")
# print("===================")
# print("===================")
# print(len(forest.sample_clusters))
# print(len(forest.leaf_clusters))

In [ ]:
# from sklearn.manifold import TSNE
# from scipy.spatial.distance import squareform,pdist

# total_encoding = forest.node_sample_encoding(forest.nodes())

# total_cell_sort = dendrogram(linkage(total_encoding,metric='cos',method='average'),no_plot=True)['leaves']
# node_sort = dendrogram(linkage(total_encoding.T,metric='cos',method='average'),no_plot=True)['leaves']

# plt.figure(figsize=(10,10))
# plt.imshow(total_encoding[total_cell_sort].T[node_sort].T,aspect='auto',cmap='binary')
# plt.show()

# sister_encoding = forest.node_sister_encoding(forest.nodes(depth=4))

# sister_sort = dendrogram(linkage(sister_encoding.T,metric='cos',method='average'),no_plot=True)['leaves']


# distance_matrix = squareform(pdist(sister_encoding.T,metric='jaccard'))

# plt.figure(figsize=(10,10))
# plt.imshow(distance_matrix[sister_sort].T[sister_sort],aspect='auto')
# plt.show()

# distance_sort = dendrogram(linkage(distance_matrix,metric='cos',method='average'),no_plot=True)['leaves']

# plt.figure(figsize=(10,10))
# plt.imshow(distance_matrix[distance_sort].T[distance_sort],aspect='auto')
# plt.show()

# total_encoding = forest.node_sample_encoding(forest.nodes())

# tc = TSNE(metric='jaccard').fit_transform(total_encoding.T)

# plt.figure(figsize=(10,10))
# plt.scatter(tc[:,0],tc[:,1],s=.5)
# plt.show()


## Split Clustering <a class="anchor" id="split_clustering"></a>

In [ ]:
forest.reset_clusters()
forest.interpret_splits(depth=5,sub=.5,k=20,mode='additive_mean',relatives=True,distance='cos',pca=100,override=True)
# forest.interpret_splits(depth=5,k=10,subsample=5500,mode='gain',steps=50,distance='cos',override=False,verbose=True)
# split_order = np.argsort(forest.split_labels)
# print(np.sum(forest.split_labels < 7))
# print(np.sum(forest.split_labels < 8))
# print(np.sum(forest.split_labels < 9))

In [ ]:
tree = forest.most_likely_tree(depth=5)
tree

# tree = forest.maximum_spanning_tree(depth=4)
# tree

# transitions = forest.split_cluster_transition_matrix()
# transitions[:,88]

# forest.likely_tree = forest.maximum_tree

tree


In [ ]:
forest.html_tree_summary()

In [ ]:
# forest.coordinates(override=False)
# forest.tsne_encoding(override=True)
# forest.plot_split_clusters()

# feature = "Cd34"
# coordinates = forest.coordinates(override=False,no_plot=True,pca=True)
# fi = forest.truth_dictionary.feature_dictionary[feature]
# plt.figure()
# plt.title(f"Expression of {feature}")
# plt.scatter(coordinates[:,0],coordinates[:,1],c=forest.output[:,fi],s=10)
# plt.colorbar()
# plt.show()

## Sample Clustering <a class="anchor" id="sample_clustering"></a>

In [ ]:
forest.tsne(pca=100,override=True)

In [ ]:
# forest.reset_sample_clusters()
# forest.cluster_samples_encoding(sub=.5,k=20,metric='cosine',pca=50,override=True)
# forest.cluster_samples_simple(pca=True,k=20,steps=50,override=True,verbose=True)
forest.plot_sample_clusters()


In [ ]:

tc = forest.coordinates(no_plot=True)

for split_cluster in forest.split_clusters:
    print(split_cluster.id)
    try:
        print(sorted(list(split_cluster.braid_features().items()),key=lambda x:x[1])[::-1])
#         print(list(split_cluster.braid_features().items()))
    except:
        pass
    split_cluster.plot_cell_counts()

    sister_color = split_cluster.sister_scores()

    f = plt.figure(figsize=(15,10))
    plt.title("Sister Split")
    plt.scatter(tc[:,0],tc[:,1],c=sister_color,cmap='bwr',vmin=max(sister_color)*-1)
    plt.colorbar()
    plt.show()


In [ ]:
# cluster_matrix = np.zeros((len(forest.sample_clusters),len(forest.leaf_clusters)))
# for i,sample_cluster in enumerate(forest.sample_clusters):
#     leaf_cluster_frequency = sample_cluster.leaf_cluster_frequency(plot=False)[1]
#     for j,lcf in enumerate(leaf_cluster_frequency):
#         cluster_matrix[i,j] = lcf
        
# plt.figure(figsize=(20,10))
# plt.title("Occurrence of Leaf Clusters in Cell Clusters",fontsize=30)
# plt.imshow(np.log(cluster_matrix+1),aspect='auto')
# plt.xlabel("Leaf Clusters",fontsize=24)
# plt.ylabel("Cell Clusters",fontsize=24)
# plt.colorbar()
# plt.show()

# vertical_order = dendrogram(linkage(cluster_matrix,metric='correlation'),no_plot=True)['leaves']
# horizontal_order = dendrogram(linkage(cluster_matrix.T,metric='correlation'),no_plot=True)['leaves']

# rearranged_sample_ticks = np.arange(len(forest.sample_clusters))[vertical_order]
# rearranged_leaf_ticks = np.arange(len(forest.leaf_clusters))[horizontal_order]

# plt.figure(figsize=(20,10))
# plt.title("Occurrence of Leaf Clusters in Cell Clusters",fontsize=30)
# plt.imshow(np.log(cluster_matrix+1)[vertical_order].T[horizontal_order].T,aspect='auto')
# plt.xlabel("Leaf Clusters",fontsize=24)
# plt.ylabel("Cell Clusters",fontsize=24)
# plt.xticks(np.arange(len(rearranged_leaf_ticks)),rearranged_leaf_ticks)
# plt.yticks(np.arange(len(rearranged_sample_ticks)),rearranged_sample_ticks)
# plt.colorbar()
# plt.show()

## Louvain Comparison

In [ ]:
import community
import networkx as nx
from sklearn.neighbors import kneighbors_graph

In [ ]:
# Performed per instructions suggested in python-louvain
plt.figure()
sg = kneighbors_graph(counts,metric='cosine',n_neighbors=5)
g = nx.from_scipy_sparse_matrix(sg)
partition = community.best_partition(g,resolution=.5)

In [ ]:
size = float(len(set(partition.values())))
pos = nx.spring_layout(g)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(g, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


In [ ]:
louvain_raw_labels = [partition[i] for i in range(len(forest.samples))]
set(louvain_raw_labels)

In [ ]:
ig = community.induced_graph(partition,g)
ig_pos = nx.spring_layout(ig)
nx.draw_networkx_nodes(ig, ig_pos)
nx.draw_networkx_edges(ig, ig_pos)

In [ ]:
plt.figure()
plt.scatter(forest.coordinates(no_plot=True)[:,0],forest.coordinates(no_plot=True)[:,1],c=louvain_raw_labels,cmap='rainbow',s=1)
plt.show()

In [ ]:
## Now let's try training Louvain on the forest leaf sample encoding:

plt.figure()
sg = kneighbors_graph(forest.node_sample_encoding(forest.leaves()),metric='jaccard',n_neighbors=5)
g = nx.from_scipy_sparse_matrix(sg)
partition = community.best_partition(g,resolution=.5)


In [ ]:
size = float(len(set(partition.values())))
pos = nx.spring_layout(g)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(g, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


In [ ]:
louvain_forest_labels = [partition[i] for i in range(len(forest.samples))]
set(louvain_raw_labels)

In [ ]:
plt.figure()
plt.scatter(forest.coordinates(no_plot=True)[:,0],forest.coordinates(no_plot=True)[:,1],c=louvain_forest_labels,cmap='rainbow',s=1)
plt.show()

In [ ]:
# from sklearn.cluster import DBSCAN
# from hdbscan import HDBSCAN

# encoding = forest.node_sample_encoding(forest.leaves())

# d_cell_clusters = HDBSCAN(min_samples=5).fit_predict(encoding)
# d_leaf_clusters = HDBSCAN(min_samples=5).fit_predict(encoding.T)

# d_cell_ordering = np.argsort(d_cell_clusters)
# d_leaf_ordering = np.argsort(d_leaf_clusters)
# # d_leaf_ordering = dendrogram(linkage(encoding.T,metric='cos',method='average'),no_plot=True)['leaves']


# plt.figure(figsize=(20,8))
# plt.imshow(encoding[d_cell_ordering].T[d_leaf_ordering].T,aspect='auto',cmap='binary')
# plt.show()

# gain_matrix = forest.local_gain_matrix(forest.nodes())

# d_leaf_clusters = HDBSCAN(min_samples=5).fit_predict(gain_matrix.T)
# d_leaf_ordering = np.argsort(d_leaf_clusters)

# plt.figure(figsize=(10,10))
# plt.imshow(gain_matrix[feature_sort].T[d_leaf_ordering],aspect='auto',)
# plt.show()

# print(set(d_cell_clusters))

In [ ]:
# d_p_cell_clusters = HDBSCAN(min_samples=5).fit_predict(training_counts)
# d_p_feature_clusters = HDBSCAN(min_samples=5).fit_predict(training_counts.T)

# d_p_cell_ordering = np.argsort(d_p_cell_clusters)
# # d_p_feature_ordering = np.argsort(d_p_feature_clusters)

# plt.figure(figsize=(20,8))
# plt.imshow(training_counts[d_p_cell_ordering],aspect='auto',cmap='binary')
# plt.show()

# print(set(d_p_cell_clusters))

In [ ]:
# from sklearn.cluster import KMeans

# k_cell_clusters = KMeans(n_clusters=10).fit_predict(counts)

# k_cell_ordering = np.argsort(k_cell_clusters)

# plt.figure(figsize=(15,10))
# plt.title("Cell x Gene Expression, K-Means",fontsize=20)
# plt.imshow(counts[k_cell_ordering],aspect='auto')
# plt.xlabel("Genes",fontsize=15)
# plt.ylabel("Cells",fontsize=15)
# plt.colorbar()
# plt.show()

## Evaluating Prediction Error <a class="anchor" id="prediction_error"></a>

In [ ]:
test_counts = forest.test
# prediction = forest.predict_matrix(test_counts)
prediction_features = prediction[:,:4290]

plt.figure()
plt.title("Distribution of Values in Test Counts")
plt.hist(test_counts.flatten(),bins=50,log=True)
plt.ylim(0,1e6)
plt.show()

plt.figure()
plt.title("Distribution of Values in Prediction")
plt.hist(prediction.flatten(),bins=50,log=True)
plt.ylim(0,1e6)
plt.show()

plt.figure()
plt.title("Distribution of Error")
plt.hist((test_counts - prediction_features).flatten(),bins=50,log=True)
plt.ylim(0,1e6)
plt.show()

In [ ]:
# forest.node_sample_encoding(forest.leaves())
# forest.leaves()[3].samples
# forest.trees[3].plot()
# forest.trees[3].tree_movie('./tree_movies/m1/pr1')
# forest.trees[4].tree_movie('./tree_movies/m2/pr1')

In [ ]:
zero_mask = counts == 0
zero_sum = np.sum(zero_mask,axis=0)
zero_percentrage = zero_sum / counts.shape[0]

plt.figure("sparsity_graph",figsize=(15,10))
plt.title("Sparsity of Features",fontsize=20)
plt.hist(zero_percentrage,bins=np.arange(0,1.05,0.05))
plt.xlabel("Sparsity (Fraction of values = 0)",fontsize=15)
plt.ylabel("Frequency",fontsize=15)
plt.show()

In [ ]:
random_mask = np.random.rand(*test_counts.flatten().shape) < (subsampling/10)

plt.figure("general_scatter")
plt.title("True Expression vs Predicted Expression")
plt.scatter(test_counts.flatten()[random_mask],prediction_features.flatten()[random_mask],s=1,alpha=.3)
plt.xlabel("True Expression")
plt.ylabel("Predicted Expression")
plt.savefig(output_directory+"general_scatter.png")

In [ ]:

plt.figure("mae_vs_mean",figsize=(20,4))
plt.title("Mean Absolute Error of Feature Predictions vs Mean Feature Value")
plt.scatter(np.mean(test_counts,axis=0),np.mean(np.abs(test_counts - prediction_features), axis=0),s=.1)
plt.plot([0,4],[0,4],c='r')
plt.xlabel("Mean Gene Expression")
plt.ylabel("Mean Absolute Error")
plt.ylim((0,4))
plt.savefig(output_directory+"mae_vs_mean.png")

# Calculate MAE
mae = np.mean(np.abs(test_counts - np.tile(np.mean(test_counts,axis=0),(test_counts.shape[0],1))), axis=0)

plt.figure("mean_abs_dev_vs_mean",figsize=(20,4))
plt.title("Mean Absolute Deviation of Features vs Expression Level")
plt.scatter(np.mean(test_counts,axis=0),mae,s=.1)
plt.plot([0,4],[0,4],c='r')
plt.xlabel("Mean Gene Expression")
plt.ylabel("Mean Absolute Deviation")
plt.ylim((0,4))
plt.savefig(output_directory+"mean_abs_dev_vs_mean.png")

In [ ]:
random_mask = np.random.rand(*true_counts.flatten().shape) < subsampling

plt.figure("bimodal_scatter_by_mean",figsize=(6,4))
plt.title("Observed Expressions Vs Mean Expression of Feature")
plt.scatter(np.tile(np.mean(true_counts,axis=0),(true_counts.shape[0],1)).flatten()[random_mask],true_counts.flatten()[random_mask],s=.1,alpha=.1, label="Observed Expression")
# plt.scatter(np.mean(true_counts,axis=0),np.median(true_counts,axis=0),c='r',s=.5, label="Mean Expression")
plt.xlabel("Mean Expression")
plt.ylabel("Observed Expression")
plt.xlim(0,4)
plt.legend()
plt.savefig(output_directory+"bimodal_scatter_by_mean.png")

for cluster in forest.sample_clusters:
    cluster_samples = cluster.samples
    cluster_counts = forest.output[cluster_samples]
    mean_expression = np.mean(cluster_counts,axis=0)
    print(cluster_counts.shape)
    plt.figure()
    plt.scatter(np.tile(mean_expression,(cluster_counts.shape[0],1)).flatten(),cluster_counts.flatten(),s=.1,alpha=.1)
    plt.xlim(0,4)
    plt.show()

In [ ]:
# print true_counts[:10,:10]

random_mask = np.random.rand(*test_counts.flatten().shape) < subsampling

plt.figure("bimodal_scatter_by_mean_predicted",figsize=(20,4))
plt.title("Predicted Values Vs Observed Means")
plt.scatter(np.tile(np.mean(test_counts,axis=0),(test_counts.shape[0],1)).flatten()[random_mask],prediction_features.flatten()[random_mask],s=.5,alpha=1,label="Expression")
plt.scatter(np.mean(test_counts,axis=0),np.median(test_counts,axis=0),c='m',s=1,label="Medians")
plt.scatter(np.mean(test_counts,axis=0),np.mean(test_counts,axis=0),c='c',s=1,label="Means")
plt.xlim(0,3)
plt.legend()
plt.savefig(output_directory+"bimodal_scatter_by_mean_predicted.png")


In [ ]:
random_mask = np.random.rand(*test_counts.flatten().shape) < subsampling

plt.figure("error_scatter_by_mean", figsize=(20,4))
plt.title("Observed Error vs Observed Mean of Feature")
plt.scatter(np.tile(np.mean(test_counts,axis=0),(test_counts.shape[0],1)).flatten()[random_mask],(test_counts - prediction_features).flatten()[random_mask],alpha=.3,s=.3)
plt.xlabel("Mean Expression")
plt.ylabel("Predicted Expression")
plt.savefig(output_directory+"error_scatter_by_mean.png")

In [ ]:
plt.figure("mean_error_vs_mean",figsize=(20,4))
plt.title("Mean Absolute Error Vs Mean Expression of Gene")
plt.scatter(np.mean(test_counts,axis=0),np.mean(np.abs(test_counts-prediction_features),axis=0),s=.5,c=np.std(test_counts,axis=0))
plt.colorbar(label="Standard Deviation of Feature")
# plt.plot([0,3],[0,3])
plt.ylim((0,4))
plt.xlabel("Mean Expression")
plt.ylabel("Mean Error")
plt.savefig(output_directory+"mean_error_vs_mean.png")

In [ ]:
plt.figure("mean_error_vs_std",figsize=(4,4))
plt.title("Mean Absolute Error Vs Standard Deviation of Gene Expression")
plt.scatter(np.std(test_counts,axis=0),np.mean(np.abs(test_counts-prediction_features),axis=0),s=.1)
# plt.colorbar(label="Standard Deviation of Feature")
plt.plot([0,4],[0,4])
plt.ylim((0,4))
plt.xlim((0,4))
plt.xlabel("Standard Deviation")
plt.ylabel("Mean Error")
plt.savefig(output_directory+"mean_error_vs_std.png")

In [ ]:
# from scipy.stats import pearsonr

correlations = []

for i in range(test_counts.shape[1]):
#     print(test_counts[:,i])
#     print(prediction[:,i])
#     print(pearsonr(test_counts[:,i],prediction[:,i]))
    correlations.append(pearsonr(test_counts[:,i],prediction_features[:,i])[0])
    if np.isnan(correlations[-1]):
        correlations[-1] = 0
        

print(len(correlations))

plt.figure("correlation_vs_mean",figsize=(20,4))
plt.title("Correlation of Predictions to Features per Feature")
plt.scatter(np.mean(test_counts,axis=0),correlations,s=.3,c=np.std(test_counts,axis=0))
plt.colorbar(label="Standard Deviation of Feature")
plt.xlabel("Mean Expression")
plt.ylabel("Prediction Correlation")
plt.savefig(output_directory+"correlation_vs_mean.png")

In [ ]:
# from scipy.stats import linregress

x = np.std(true_counts,axis=0)
y = np.mean(true_counts-prediction_features,axis=0)

# slope, intercept, rvalue, pvalue, std_err = linregress(x, y=y)

plt.figure("mean_error_vs_variance")
plt.title("Standard Deviation vs Mean Error")
plt.scatter(x,y,s=.1)
# plt.plot(x,x*slope + intercept, 'r', label = str(np.around(rvalue,decimals=3)))
plt.plot([0,3],[0,3])
plt.plot([0,3],[0,-3])
plt.ylim((-1,1))
plt.xlabel("Standard Deviation")
plt.ylabel("Mean Error")
plt.legend()
plt.savefig(output_directory+"mean_error_vs_variance.png")

In [ ]:
correlations = []

for i in range(true_counts.shape[1]):
    correlations.append(pearsonr(true_counts[:,i],predicted[:,i])[0])
    if isnan(correlations[-1]):
        correlations[-1] = 0
print(len(correlations))

    
plt.figure("correlation_vs_mean",figsize=(20,4))
plt.title("Correlation of Predictions vs Expression Level")
plt.scatter(correlations,np.mean(true_counts,axis=0),s=.3,c=variation(true_counts,axis=0))
plt.ylabel("Mean Expression")
plt.xlabel("Correlation")
plt.colorbar(label="Coefficient of Variation")
plt.clim(0,10)
plt.savefig(output_directory+"correlation_vs_mean_vs_cov.png",dpi=500)

In [ ]:
correlations = []

for i in range(true_counts.shape[1]):
    correlations.append(pearsonr(true_counts[:,i],predicted[:,i])[0])
    if isnan(correlations[-1]):
        correlations[-1] = 0
    
print(len(correlations))

plt.figure("correlation_vs_cov")
plt.title("Correlation of features vs Coefficient of Variation")
plt.xlabel("Coefficient of Variation")
plt.ylabel("Correlation")
plt.scatter(variation(true_counts,axis=0),correlations,s=.1,c=np.std(true_counts,axis=0))
plt.colorbar(label="Standard Deviation of Feature")
plt.xlim(0,2)
plt.show()

In [ ]:
random_mask = np.random.rand(*test_counts.flatten().shape) < (subsampling/10.0)

errors = prediction_features - true_counts

plt.figure("error_vs_true_expression")
plt.title("Error vs True Expression")
plt.scatter(test_counts.flatten()[random_mask],errors.flatten()[random_mask],s=.3,alpha=.3,c=np.tile(np.mean(test_counts,axis=0),(test_counts.shape[0],1)).flatten()[random_mask],cmap='inferno')
plt.xlabel("True Expression")
plt.ylabel("Error")
plt.colorbar(label="Mean expression of gene")
plt.plot([])
plt.savefig(output_directory+"error_vs_true_expression.png")

In [ ]:
error = prediction_features - test_counts

mean_cell_error = np.mean(error,axis=1)
mean_gene_error = np.mean(error,axis=0)

print(mean_gene_error.shape)
print(mean_cell_error.shape)

plt.figure()
plt.title("Distribution of Mean Feature Errors")
plt.hist(mean_gene_error,bins=20)
plt.xlabel("Mean Error")
plt.ylabel("Frequency")
plt.show()

plt.figure()
plt.title("Distribution of Mean Cell Errors")
plt.hist(mean_cell_error,bins=20)
plt.xlabel("Mean Error")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# expression_sorted_features = test_counts.T[np.argsort(np.mean(test_counts,axis=0))].T
# expression_sorted_prediction = prediction_features.T[np.argsort(np.mean(test_counts,axis=0))].T


plt.figure("predictability")
plt.title("Correlation of Features to Other Features, Sorted By Mean Expression")
plt.imshow(np.corrcoef(expression_sorted_features.T))
plt.colorbar(label="Correlation")
plt.show()


In [ ]:
expression_sorted_features = test_counts.T[np.argsort(np.mean(test_counts,axis=0))].T

random_mask = np.random.rand(*expression_sorted_features.flatten().shape) < subsampling


plt.figure("ranked_feature_expression",figsize=(20,4))
plt.title("Observed Expressions For Each Feature Ranked By Mean Expression")
plt.scatter(np.tile(np.arange(expression_sorted_features.shape[1]),(expression_sorted_features.shape[0],1)).flatten()[random_mask],expression_sorted_features.flatten()[random_mask],s=.3,alpha=.1, label="Observed Expression")
# plt.scatter(np.mean(true_counts,axis=0),np.median(true_counts,axis=0),c='r',s=.5, label="Mean Expression")
plt.xlabel("Feature Rank")
plt.ylabel("Observed Expression")
plt.ylim((0,10))
plt.legend()
plt.savefig(output_directory+"rank_ordered_expresssion.png")

for cluster in forest.sample_clusters:
    cluster_samples = cluster.samples
    cluster_counts = forest.output[cluster_samples].T[np.argsort(np.mean(forest.output,axis=0))].T
    print(cluster_counts.shape)
    plt.figure(figsize=(20,4))
    plt.scatter(np.tile(np.arange(cluster_counts.shape[1]),(cluster_counts.shape[0],1)).flatten(),cluster_counts.flatten(),s=.3,alpha=.1)
    plt.show()

In [ ]:
agglomerated_features = test_counts.T[feature_sort].T
random_mask = np.random.rand(*agglomerated_features.flatten().shape) < subsampling

plt.figure("ranked_feature_expression",figsize=(20,4))
plt.title("Observed Expressions For Each Feature Ranked By Mean Expression")
plt.scatter(np.tile(np.arange(agglomerated_features.shape[1]),(agglomerated_features.shape[0],1)).flatten()[random_mask],agglomerated_features.flatten()[random_mask],s=.3,alpha=.1, label="Observed Expression")
plt.xlabel("Feature Rank")
plt.ylabel("Observed Expression")
plt.ylim((0,10))
plt.legend()
plt.savefig(output_directory+"rank_ordered_expresssion.png")



In [ ]:
# def quick_hetorskedasticity(feature):
#     sorted_feature = sorted(feature)
#     first = sorted_feature[:int(len(sorted_feature)/2)]
#     second = sorted_feature[int(len(sorted_feature)/2):]
#     fv = np.var(first)
#     sv = np.var(second)
#     return fv,sv

# expression_sorted_features.shape

# # random_mask = np.random.rand(*expression_sorted_features.flatten().shape) < subsampling

# heteroskedasticity = [quick_hetorskedasticity(feature)[0]/quick_hetorskedasticity(feature)[1] for feature in expression_sorted_features.T]
# var1 = [quick_hetorskedasticity(feature)[0] for feature in expression_sorted_features.T]
# var2 = [quick_hetorskedasticity(feature)[1] for feature in expression_sorted_features.T]

# print(var1)
# print(var2)

# # print(len(heteroskedasticity))

# # print(heteroskedasticity)

# plt.figure("heteroskedasticity",figsize=(20,4))
# plt.title("Heteroskedasticity",fontsize=20)
# plt.scatter(np.arange(0,len(var1)),var1,s=4,alpha=1,c='r',label="First")
# plt.scatter(np.arange(0,len(var2)),var2,s=4,alpha=1,c='b',label="Second")
# # plt.scatter(np.mean(true_counts,axis=0),np.median(true_counts,axis=0),c='r',s=.5, label="Mean Expression")
# plt.xlabel("Feature Rank",fontsize=15)
# plt.ylabel("Ratio\n Var for f greater than median vs less",fontsize=12)
# plt.legend()
# plt.savefig(output_directory+"heterskedasticity.png")

# plt.figure("heteroskedasticity2",figsize=(20,4))
# plt.title("Heteroskedasticity",fontsize=20)
# plt.scatter(np.arange(0,len(heteroskedasticity)),heteroskedasticity,s=4,alpha=1,c='b')
# plt.xlabel("Feature Rank",fontsize=15)
# plt.ylabel("Ratio\n Var for f greater than median vs less",fontsize=12)
# plt.savefig(output_directory+"heterskedasticity.png")

In [ ]:
# import random

# for j in range(10):
#     i = random.randint(1,true_counts.shape[0])
#     plt.figure('cell_multiplex' + str(i))
#     plt.scatter(np.arange(expression_sorted_features.shape[1]),expression_sorted_features[i],s=.1)
#     plt.show()

In [ ]:
# for j in range(10):
#     i = random.randint(1,expression_sorted_features.shape[1])
#     plt.figure("feature_dist_multiplex" + str(i))
#     plt.hist(expression_sorted_features[:,i],bins=50,log=True)
#     plt.show()

# # deciles = np.zeros((20,expression_sorted_features.shape[1]))
# # derivatives = np.zeros((19,expression_sorted_features.shape[1]))
# # for i,feature in enumerate(expression_sorted_features.T):
# #     bins,edges = np.histogram(feature,bins=20,range=(np.min(feature),np.max(feature)))
# # #     print(bins)
# # #     print(edges)
# # #     print(np.array([y-x for x,y in zip(bins,bins[1:])]))
# #     deciles[:,i] = bins
# #     derivatives[:,i] = np.array([y-x for x,y in zip(bins,bins[1:])])

# # plt.figure(figsize=(20,4))
# # plt.imshow(deciles,aspect='auto')
# # plt.show()

# # plt.figure(figsize=(20,4))
# # plt.imshow(derivatives,aspect='auto')
# # plt.show()

In [ ]:
# print true_counts[:10,:10]

# expression_sorted_features = test_counts.T[np.argsort(np.mean(test_counts,axis=0))].T

# expression_sorted_prediction = prediction.T[np.argsort(np.mean(test_counts,axis=0))].T

# random_mask = np.random.rand(*expression_sorted_features.flatten().shape) < (subsampling)

# plt.figure("ranked_feature_predicted",figsize=(20,4))
# plt.title("Predicted Values By Mean Expression Ranking")
# plt.scatter(np.tile(np.arange(expression_sorted_features.shape[1]),(expression_sorted_features.shape[0],1)).flatten()[random_mask],expression_sorted_prediction.flatten()[random_mask],s=.05,alpha=.3,label="Expression",c=np.abs(expression_sorted_prediction-expression_sorted_features).flatten()[random_mask],cmap='plasma')
# # plt.scatter(np.arange(expression_sorted_features.shape[1]),np.median(expression_sorted_features,axis=0),c='m',s=.5,label="Medians")
# # plt.scatter(np.arange(expression_sorted_features.shape[1]),np.mean(expression_sorted_features,axis=0),c='c',s=.5,label="Means")
# plt.legend()
# plt.colorbar(label="Absolute Value of Error")
# plt.clim(0,10)
# plt.savefig(output_directory+"rank_ordered_predicted.png",dpi=500)

In [ ]:
# random_mask = np.random.rand(*expression_sorted_features.flatten().shape) < subsampling

# plt.figure("error_scatter_ranked", figsize=(20,4))
# plt.title("Observed Error By Mean Expression Ranking")
# plt.scatter(np.tile(np.arange(expression_sorted_features.shape[1]),(expression_sorted_features.shape[0],1)).flatten()[random_mask],(expression_sorted_features - expression_sorted_prediction).flatten()[random_mask],alpha=.1,s=.1)
# plt.xlabel("Rank")
# plt.ylabel("Predicted Expression")
# plt.savefig(output_directory+"rank_ordered_error.png")

In [ ]:
# correlations = []

# for i in range(test_counts.shape[1]):
    
#     correlations.append(pearsonr(test_counts[:,i],prediction[:,i])[0])
#     if isnan(correlations[-1]):
#         correlations[-1] = 0
        
# print(len(correlations))
# print(test_counts.shape)
        
# plt.figure("correlation_vs_mean",figsize=(20,4))
# plt.title("Correlation of Predictions per Expression Ranked Feature")
# plt.scatter(np.arange(len(correlations)),np.array(correlations)[np.argsort(np.mean(test_counts,axis=0))],s=.5)
# plt.xlabel("Feature Mean Expression Rank")
# plt.ylabel("Prediction Correlation")
# plt.savefig(output_directory+"ranked_correlation.png")


In [ ]:
# import random
 
# j = 0

# while j < 10:
#     i = random.randint(1,expression_sorted_features.shape[1])
#     if pearsonr(expression_sorted_features[:,i],expression_sorted_prediction[:,i])[0] < .5:
#         continue
#     plt.figure('feature_pred_multiplex' + str(i))
#     plt.gca().axis('equal')
#     plt.title("Mean Expresion: " + str(np.mean(expression_sorted_features[:,i])) + " Corr: " + str(pearsonr(expression_sorted_features[:,i],expression_sorted_prediction[:,i])[0]))
#     plt.scatter(expression_sorted_features[:,i],expression_sorted_prediction[:,i],s=1)
#     plt.xlabel("True Expression")
#     plt.ylabel("Predicted Expression")
#     plt.show()
    
#     j += 1

In [ ]:
# correlations = []

# for i in range(expression_sorted_features.shape[1]):
    
#     correlations.append(pearsonr(expression_sorted_features[:,i],expression_sorted_prediction[:,i])[0])
#     if isnan(correlations[-1]):
#         correlations[-1] = 0
    
# print(len(correlations))
# print(correlations[:10])
# print(np.arange(.05,1,.05))
    
# plt.figure("correlation_histogram")
# plt.title("Distribution of Feature Correlations")
# plt.hist(correlations,bins=np.arange(0,1,.05))
# plt.savefig(output_directory+"correlation_distributions.png")

In [ ]:
# pearsonr(true_counts.flatten(),imputed_gradient.flatten())

In [ ]:
# plt.figure()
# plt.title("Median Random Forest Prediction")
# plt.xlabel("Truth")
# plt.ylabel("Imputed")
# plt.scatter(truth,pred_basic_forest,s=.003)
# plt.plot([0,15],[0,15])
# plt.savefig('figures/basic_error_scatter.png')

In [ ]:
# print np.var(predicted,axis=0).shape

# plt.figure("means")
# plt.title("Distribution of Means")
# plt.hist(np.mean(true_counts,axis=0),bins=np.arange(20),alpha=.1,label="Data")
# plt.show()

# plt.figure("stds")
# plt.title("Distribution of Standard Deviations")
# plt.hist(np.std(true_counts,axis=0), bins=np.arange(20),alpha=.1,label="Data")
# plt.show()

# plt.figure("naive_mae")
# plt.title("Distribution of Means of Error")
# plt.hist(np.mean(np.abs(true_counts - predicted), axis=0),bins=np.arange(20),alpha=.1,label="Error")
# plt.show()

# plt.figure("naive_mse")
# plt.title("Distribution of MSE per feature")
# plt.hist(np.mean((true_counts - predicted)**2, axis=0),bins=np.arange(20),alpha=.1,label="Error")
# plt.show()

# np.random.shuffle(predicted)
# plt.figure("mae_shuffled")
# plt.title("Distribution of Means of Error (Shuffled)")
# plt.hist(np.var(np.abs(true_counts - predicted),axis=0),bins=np.arange(20),alpha=.1,label="Shuffled")
# # plt.legend()
# plt.show()


In [ ]:
# plt.figure()
# plt.title("Gradient Random Forest Prediction")
# plt.xlabel("Truth")
# plt.ylabel("Imputed")
# plt.scatter(truth,pred_gradient,s=.003)
# plt.plot([0,15],[0,15])
# plt.savefig('figures/gradient_error_scatter.png')

In [ ]:
# print np.sum(true_counts,axis=1).shape

# plt.figure("cell_histograms")
# plt.title("Frequency of Total Cell Counts")
# plt.hist(np.sum(true_counts,axis=1))

# plt.show()

In [ ]:
# power = 2

# print(np.sum(true_counts,axis=1).shape)

# plt.figure("cell_histograms")
# plt.title("Frequency of Total Cell Counts (Log)")
# plt.hist(np.sum(np.power(true_counts,power),axis=1))
# plt.show()

In [ ]:
# plt.figure()
# plt.title("KNN Imputation")
# plt.xlabel("Truth")
# plt.ylabel("Imputed")
# plt.scatter(truth,pred_knn,s=.003)
# plt.plot([0,15],[0,15])
# plt.savefig('figures/knn_error_scatter.png')

In [ ]:
# plt.figure()
# plt.title("Soft Imputation Prediction")
# plt.xlabel("Truth")
# plt.ylabel("Imputed")
# plt.scatter(truth,pred_soft,s=.003)
# plt.plot([0,15],[0,15])
# plt.savefig('figures/soft_error_scatter.png')

In [ ]:
# plt.figure()
# plt.title("Sklearn Random Forest Prediction")
# plt.xlabel("Truth")
# plt.ylabel("Imputed")
# plt.scatter(truth,pred_builtin,s=.003)
# plt.plot([0,15],[0,15])
# plt.savefig('figures/sklearn_rf_error_scatter.png')

In [ ]:
# true_iris = np.loadtxt('./testing/iris.trunc')
# dropped_iris = np.loadtxt('./testing/iris.drop')
# nan_iris = dropped_iris.copy()
# nan_iris[nan_iris == 0] = np.nan

# forest_prediction = np.loadtxt('./iris_test/run.prediction')

In [ ]:
# soft_iris = fi.SoftImpute().complete(nan_iris)

# knn_iris = fi.KNN(k=15).complete(nan_iris)

# factorized_iris = fi.MatrixFactorization().complete(nan_iris)

# iterative_iris = fi.IterativeSVD(rank=5).complete(nan_iris)

In [ ]:

# guess_mask = dropped_iris.flatten() == 0

# truth = true_iris.flatten()[guess_mask] 

# print "Pearson correlations of Soft, KNN, Factorized, and Forest"
# print pearsonr(soft_iris.flatten()[guess_mask],truth)
# print pearsonr(knn_iris.flatten()[guess_mask],truth)
# print pearsonr(factorized_iris.flatten()[guess_mask],truth)
# print pearsonr(forest_prediction.flatten()[guess_mask],truth)

# print "MSE of Soft, KNN, Factorized, and Forest"
# print np.mean(((soft_iris.flatten()[guess_mask] - truth) ** 2))
# print np.mean(((knn_iris.flatten()[guess_mask] - truth) ** 2))
# print np.mean(((factorized_iris.flatten()[guess_mask] - truth) ** 2))
# print np.mean(((forest_prediction.flatten()[guess_mask] - truth) ** 2))


In [ ]:
# plt.figure("soft_iris")
# plt.title("Soft Imptuation of the Iris Dataset")
# plt.xlabel("Imputed")
# plt.ylabel("True")
# plt.scatter(soft_iris.flatten()[guess_mask],true_iris.flatten()[guess_mask],s=1)
# plt.plot([0,8],[0,8])
# plt.tight_layout()
# plt.savefig("figures/soft_iris_scatter.png")

# plt.figure("knn_iris")
# plt.title("KNN Imptuation of the Iris Dataset")
# plt.xlabel("Imputed")
# plt.ylabel("True")
# plt.scatter(knn_iris.flatten()[guess_mask],true_iris.flatten()[guess_mask],s=1)
# plt.plot([0,8],[0,8])
# plt.tight_layout()
# plt.savefig("figures/knn_iris_scatter.png")

# plt.figure("factorized_iris")
# plt.title("Matrix Factorization Imptuation of the Iris Dataset")
# plt.xlabel("Imputed")
# plt.ylabel("True")
# plt.scatter(factorized_iris.flatten()[guess_mask],true_iris.flatten()[guess_mask],s=1)
# plt.plot([0,8],[0,8])
# plt.tight_layout()
# plt.savefig("figures/factorized_iris_scatter.png")

# plt.figure("forest_iris")
# plt.title("Median Forest (Non-Gradient) Imptuation of the Iris Dataset")
# plt.xlabel("Imputed")
# plt.ylabel("True")
# plt.scatter(forest_prediction.flatten()[guess_mask],true_iris.flatten()[guess_mask],s=1)
# plt.plot([0,8],[0,8])
# plt.tight_layout()
# plt.savefig("figures/forest_iris_scatter.png")

In [ ]:
# nan_counts = held_out_counts.copy()
# nan_counts[nan_counts == 0] = np.nan

# soft_blood = fi.SoftImpute().complete(nan_counts)

# knn_blood = fi.KNN(k=15).complete(nan_counts)

In [ ]:
# mask = mask.astype(dtype=bool)
# # mask = np.logical_and(true_counts != 0, mask)

# truth = true_counts[mask].flatten()
# pred_basic_forest = imputed_basic[mask].flatten()
# pred_gradient = imputed_gradient[mask].flatten()

# pred_builtin = imputed_builtin[mask].flatten()

# pred_knn = knn_blood[mask].flatten()
# pred_soft = soft_blood[mask].flatten()

# print "Pearson of random, gradient, builtin, knn, and soft"

# print pearsonr(truth,pred_basic_forest)
# print pearsonr(truth,pred_gradient)
# print pearsonr(truth,pred_builtin)
# print pearsonr(truth,pred_knn)
# print pearsonr(truth,pred_soft)

# print "MSE of random, gradient, builtin, knn, and soft"

# print np.mean((pred_basic_forest - truth) ** 2)
# print np.mean((pred_gradient - truth) ** 2)
# print np.mean((pred_builtin - truth) ** 2)
# print np.mean((pred_knn - truth) ** 2)
# print np.mean((pred_soft - truth) ** 2)

# print "Mean absolute error"
# print np.mean(np.abs(pred_basic_forest - truth))
# print np.mean(np.abs(pred_gradient - truth))
# print np.mean(np.abs(pred_builtin - truth))
# print np.mean(np.abs(pred_knn - truth))
# print np.mean(np.abs(pred_soft - truth))


In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import train_test_split
# from scipy.stats import pearsonr

# import time

# start_time = time.time()

# for i in range(10):

#     print "Selecting features"
    
#     in_features = np.random.rand(true_counts.shape[1]) > .7
#     out_features = np.logical_not(in_features)

#     training_samples = np.random.rand(true_counts.shape[0]) > .3
#     testing_samples = np.logical_not(training_samples)

#     training_in = true_counts[training_samples].T[in_features].T
#     training_out = true_counts[training_samples].T[out_features].T

#     testing_in = true_counts[testing_samples].T[in_features].T
#     testing_out = true_counts[testing_samples].T[out_features].T

#     print "Initializing feature"
    
#     forest = GradientBoostingRegressor()

#     print "Fitting"
    
#     forest.fit(training_in,training_out)
    
#     print "Predicting"
    
#     prediction = forest.predict(testing_in)

#     print np.sum(in_features)
#     print np.sum(out_features)
#     print np.sum(training_samples)
#     print np.sum(testing_samples)
    
#     print prediction.shape
#     print testing_out.shape
    
#     print "Pearson R"
#     print pearsonr(testing_out.flatten(),prediction.flatten())
    
#     print "MSE"
#     print np.mean((prediction.flatten() - testing_out.flatten()) ** 2)
    
#     print "MAE"
#     print np.mean(np.abs(prediction.flatten() - testing_out.flatten()))

    
# print "Fitting done"

# end_time = time.time()

# print start_time - end_time

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from scipy.stats import pearsonr

# import time

# start_time = time.time()

# for i in range(10):

#     print "Selecting features"
    
#     in_features = np.random.rand(true_counts.shape[1]) > .7
#     out_features = np.logical_not(in_features)

#     training_samples = np.random.rand(true_counts.shape[0]) > .3
#     testing_samples = np.logical_not(training_samples)

#     training_in = true_counts[training_samples].T[in_features].T
#     training_out = true_counts[training_samples].T[out_features].T

#     testing_in = true_counts[testing_samples].T[in_features].T
#     testing_out = true_counts[testing_samples].T[out_features].T

#     print "Initializing feature"
    
#     forest = RandomForestRegressor(n_estimators=100,min_samples_split=50,n_jobs=int(10),max_features=400)

#     print "Fitting"
    
#     forest.fit(training_in,training_out)
    
#     print "Predicting"
    
#     prediction = forest.predict(testing_in)

#     print np.sum(in_features)
#     print np.sum(out_features)
#     print np.sum(training_samples)
#     print np.sum(testing_samples)
    
#     print prediction.shape
#     print testing_out.shape
    
#     print "Pearson R"
#     print pearsonr(testing_out.flatten(),prediction.flatten())
    
#     print "MSE"
#     print np.mean((prediction.flatten() - testing_out.flatten()) ** 2)
    
#     print "MAE"
#     print np.mean(np.abs(prediction.flatten() - testing_out.flatten()))

    
# print "Fitting done"http://localhost:8888/notebooks/evaluate_prediction.ipynb#

# end_time = time.time()

# print start_time - end_time

In [ ]:
# braid = trbr.IHMM.reconstitute('./forest_vision_braid')
# braid = trbr.IHMM(forest,beta=100,gamma=100,start_states=100,alpha_e=.5,beta_e=.5,inf_check=False)

In [ ]:
# for i in range(300):
#     print("********")
#     print(i)
#     print(braid.alpha)
#     print(braid.beta)
#     print(braid.gamma)
#     print(len(braid.hidden_states))
#     print("********")
#     braid.sample_states()

In [ ]:
# for i in range(100):
#     print("********")
#     print(f"SWEEP:{i}")
#     print(f"Beta:{braid.beta}")
#     print(f"Gamma:{braid.gamma}")
#     braid.sweep()
#     braid.max_likelihood_sweep()
#     if i%3 == 0:
#         braid.max_likelihood_sweep()
#     print(braid.oracle_transition_counts)
#     print(braid.transition_counts)


In [ ]:
# print(np.sum(braid.state_masks,axis=1))
# print(braid.state_sample_log_odds.shape)
# # print(list(braid.state_sample_log_odds[1]))
# # print(list(braid.state_raw_emission_counts[1]))
# # print(list(enumerate(np.sum(braid.state_masks,axis=1))))
# print(np.arange(12550)[braid.state_masks[9]])
# ni = 8720
# print(braid.state_log_odds_given_divergence[:,ni])
# print(braid.state_log_odds_given_child_l[:,ni])
# print(braid.state_log_odds_given_child_r[:,ni])
# print(braid.state_log_odds[:,ni])

In [ ]:
# print(sorted(enumerate(np.mean(braid.state_log_odds_given_divergence[:,braid.live_mask],axis=0)),key=lambda x: x[1]))

# ni = 98
# print(braid.state_log_odds_given_divergence[:,ni])
# print(braid.state_log_odds_given_child_l[:,ni])
# print(braid.state_log_odds_given_child_r[:,ni])
# print(braid.state_log_odds[:,ni])

# print(braid.state_log_odds[:,ni])

In [ ]:
# braid.node_states.shape

In [ ]:
# for i in range(100):
#     braid.sweep()
#     if i%5 == 0:
#         braid.max_likelihood_sweep()
# list(braid.state_sample_log_odds[1])
# np.sum(np.abs(braid.state_sample_log_odds[1]))

In [ ]:
# iftc = forest.tsne_encoding()
# iftc = forest.tsne(override=True)

# cell_id_header = np.loadtxt('/Users/boris/taylor/vision/python_prototype/raw_data/vision_sc/cell_id_header.txt',dtype='str')
# cell_ids = np.loadtxt('/Users/boris/taylor/vision/python_prototype/raw_data/vision_sc/cell_identity.txt',dtype=bool)

# mep_index = list(cell_id_header).index('MEP_narrow')

# print(mep_index)
# print(np.sum(cell_ids[:,mep_index]))

# plt.figure()
# plt.scatter(iftc[:,0],iftc[:,1],c=cell_ids[:,mep_index])
# plt.show()

In [ ]:
# plt.figure()
# plt.title("State Transition Frequency,0 is Root")
# plt.imshow(forest.split_cluster_transition_matrix(depth=4),cmap='binary')
# plt.show()
  
# # print(braid.state_raw_sample_odds[1])
# for hidden_state in range(braid.hidden_states):
#     print(np.sum(braid.state_masks[hidden_state]))
#     plt.figure(figsize=(5,5))
#     plt.title(f"Hidden State {hidden_state}",fontsize=20)
#     plt.scatter(iftc[:,0],iftc[:,1],c=braid.lr_finite(hidden_state),cmap='bwr',s=.3)
#     plt.ylim(-30,30)
#     plt.xlim(-30,40)
# #     plt.colorbar()
# #     plt.scatter(iftc[:,0],iftc[:,1],c=braid.state_raw_sample_odds[hidden_state],s=1,cmap='PuOr')
#     plt.show()
        


In [ ]:
# plt.figure()
# plt.title("Frequency of Transitions from One Hidden State To Another")
# plt.imshow(braid.pad_root_transitions(braid.raw_transition_counts()).T,cmap='binary')
# plt.show()

In [ ]:
# forest.plot_sample_feature_split(braid.lr_finite(14),plot_n=10)

In [ ]:
# braid.backup('./forest_vision_braid')


In [ ]:
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(7))[:20])
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(8))[:20])


In [ ]:
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(3))[:20])
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(4))[:20])

In [ ]:
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(13))[:20])
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(14))[:20])

In [ ]:
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(11))[:20])

In [ ]:
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(5))[:20])
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(6))[:20])

In [ ]:
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(1))[:20])
# print(forest.node_feature_summary(braid.hidden_state_to_nodes(2))[:20])

In [ ]:
# for f in forest.node_feature_summary(braid.hidden_state_to_nodes(9)):
#     print(f[0])

In [ ]:
# from goatools.base import download_go_basic_obo
# obo_fname = download_go_basic_obo()

# from goatools.base import download_ncbi_associations
# gene2go = download_ncbi_associations()


# from goatools.obo_parser import GODag

# obodag = GODag("go-basic.obo")

# from goatools.associations import read_ncbi_gene2go,dnld_assc

# geneid2gos_mouse = read_ncbi_gene2go("gene2go", taxids=[10090])

# print("{N:,} annotated mouse genes".format(N=len(geneid2gos_mouse)))

# from goatools.go_enrichment import GOEnrichmentStudy
# from goatools.test_data.genes_NCBI_10090_ProteinCoding import GENEID2NT as GeneID2nt_mus



In [ ]:
# goeaobj = GOEnrichmentStudy(
#         GeneID2nt_mus.keys(), # List of mouse protein-coding genes
#         geneid2gos_mouse, # geneid/GO associations
#         obodag, # Ontologies
#         propagate_counts = False,
#         alpha = 0.05, # default significance cut-off
#         methods = ['fdr_bh']) # defult multipletest correction method

In [ ]:
# symbol2id = {v[5]:k for k,v in GeneID2nt_mus.items()}

# def try_symbol(symbols):
#     ids = []
#     for symbol in symbols:
#         if symbol in symbol2id:
#             ids.append(symbol2id[symbol])
#     return ids

In [ ]:
# gores = goeaobj.run_study(try_symbol([x[0] for x in forest.node_feature_summary(braid.hidden_state_to_nodes(13))]))

# gores_filtered = sorted([r for r in gores if r.p_fdr_bh < 0.05],key=lambda r: r.p_fdr_bh)

# gores_filtered = [g for g in gores_filtered if g.goterm.namespace == 'biological_process' or g.goterm.namespace == 'molecular_function']

# print(gores_filtered)

In [ ]:
# gores_filtered[1].__dict__

In [ ]:
# [go.name for go in gores_filtered]

In [ ]:
# for term in [go.name for go in gores_filtered]:
#     print(term)

In [ ]:
from matplotlib.cm import get_cmap
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

rainbow=get_cmap('rainbow')

cluster_names = ["None",] + list(cell_identity_header[9:-1])
cluster_names = np.array([cluster_names]).T
clusters = sorted(list(set(cell_identity)))
clusters = np.array([clusters]).T
colors = np.array([rainbow(c/len(clusters)) for c in clusters]).T.T

print(cluster_names)
print(clusters)
print(colors)

# coordinates = forest.coordinates(type='tsne',pca=True,override=False)
coordinates = TSNE().fit_transform(PCA(n_components=2).fit_transform(counts))
fig = plt.figure(figsize=(15,10))
scatter_ax = fig.add_axes([0,0,1,1])
scatter_ax.set_title("Mouse Hematopoietic Cells By Type, FACS",color='w')
scatter_ax.scatter(coordinates[:,0],coordinates[:,1],c=cell_identity,cmap='rainbow',s=20)
scatter_ax.set_xticks([])
scatter_ax.set_yticks([])
table_ax = fig.add_axes([.05,.05,.1,.3])
table_ax.axis("off")
table_ax.table(cellText=cluster_names,cellColours=colors,bbox=[0,0,1,1],)
plt.show()

colors.shape

In [ ]:
for i in range(cell_x_split.shape[0]):
    for j in range(cell_x_split.shape[1]):
        print((i,j))
        cell_id_mask = np.array(cell_identity) == j
        print(np.sum(cell_id_mask))
        cluster_cell_scores = np.array(forest.split_clusters[i].cell_scores())
#         print(cluster_cell_scores)
#         print(cell_id_mask)
#         print(np.array(cluster_cell_scores)[np.array(cell_id_mask)])
        mean_cluster_score = np.mean(np.array(cluster_cell_scores)[np.array(cell_id_mask)])
#         print(mean_cluster_score)
        cell_x_split[i,j] = mean_cluster_score
        
cluster_sort = dendrogram(linkage(cell_x_split,metric='cos',method='average'),no_plot=True)['leaves']
# cluster_sort = np.argsort(np.array([c.mean_level() for c in forest.split_clusters]))
cell_id_sort = dendrogram(linkage(cell_x_split.T,metric='cos',method='average'),no_plot=True)['leaves']

plt.figure()
plt.imshow(cell_x_split[cluster_sort].T[cell_id_sort],aspect='auto',cmap='binary')
plt.xticks(np.arange(19),cluster_sort)
plt.yticks(np.arange(12), np.array(["None",] + list(cell_identity_header[9:-1]))[cell_id_sort])
plt.xlabel("RF Clusters")
plt.ylabel("FACS Profile")
plt.show()